In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import urllib.request
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import warnings

-------

In [2]:
#Import del módulo de donde se descargan los datos y se extrae.
from modules import data_download_extraction

In [5]:
#Descarga y extracción de archivos csv.

def data_download( Eastern = True, Western = True, Unzip = True):
    '''data_download( Eastern = True, Western = True, Unzip = True)
    
        This function downloads the 2006 simulated plants time series for fotovoltaic power generation. 
        It uses data provided by NREL. Downloads the files into a folder called 'data' where the extracted 
        files are also stored, if selected.
    '''
    if Eastern:
        print('Downloading eastern states data:')
        data_download_extraction.download_data('./links/eastern_states_links.csv', './')
        print('Download finished.')
    if Western:
        print('Downloading western states data:')
        data_download_extraction.download_data('./links/western_states_links.csv', './')
        print('Download finished.')
    if Unzip:
        print('Exracting data files:')
        data_download_extraction.unzip_data('./data/')
        print('Extraction finished.')

In [8]:
#Creación de los dataframes con los datos. Se generan archivos csv.

def get_plants_files_metadata( read = False, PATH = './data/Extracted/', UPV = True, DPV = True, to_csv = False):

    '''Function returns pandas dataframes with the metadata of the files which contain the time series.
       -UPV: solar plants with tracking technology.
       -DPV: solar plants without tracking technology.
    '''
    if read:
        #Si los archivos con datos geograficos ya están construidos solo se leen
        if UPV and DPV:
            data_UPV = pd.read_csv('metadata_UPV.csv')
            data_DPV = pd.read_csv('metadata_DPV.csv')
            return data_UPV, data_DPV
        if UPV and not DPV: 
            data_UPV = pd.read_csv('metadata_UPV.csv')
            return data_UPV
        if DPV and not UPV: 
            data_DPV = pd.read_csv('metadata_DPV.csv')
            return data_DPV
    else:
        if UPV and DPV:
            data_UPV = data_download_extraction.info_df_from_data( PATH , tech = 'UPV')
            data_DPV = data_download_extraction.info_df_from_data( PATH , tech = 'DPV')
            if to_csv: 
                data_UPV.to_csv('./metadata_UPV.csv', index = False)
                data_DPV.to_csv('./metadata_DPV.csv', index = False)
            return data_UPV, data_DPV
        if UPV and not DPV: 
            data_UPV = data_download_extraction.info_df_from_data( PATH , tech = 'UPV')
            if to_csv: 
                data_UPV.to_csv('./metadata_UPV.csv', index = False)
            return data_UPV
        if DPV and not UPV: 
            data_DPV = data_download_extraction.info_df_from_data( PATH , tech = 'DPV')
            if to_csv: 
                data_DPV.to_csv('./metadata_DPV.csv', index = False)
            return data_DPV

In [25]:
#Descarga y extracción de archivos shape del mapa de EEUU.

def get_usa_shapes():
    
    url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_nation_5m.zip'
    urllib.request.urlretrieve(url, './usa_shape.zip')
    url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
    urllib.request.urlretrieve(url, './usa_shape_division.zip')
    url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip'
    urllib.request.urlretrieve(url, './usa_shape_countys.zip')

    with zipfile.ZipFile('./usa_shape.zip', 'r') as zip_ref:
        zip_ref.extractall('./usa_shapes/usa_shape_nation/')
    with zipfile.ZipFile('./usa_shape_division.zip', 'r') as zip_ref:
        zip_ref.extractall('./usa_shapes/usa_shape_states/')
    with zipfile.ZipFile('./usa_shape_countys.zip', 'r') as zip_ref:
        zip_ref.extractall('./usa_shapes/usa_shape_countys/')

-----

In [ ]:
#generación de dataframes con puntos geograficos, acotados según una ventana máxima de 

def geographic_data(data_XPV_meta ):
    '''
    geographic_data(data_XPV_meta )
    
    Returns a geographic dataframe corresponding with the points of the locations on the metadata.
    '''
    #Creación de puntos geográficos a partir del dataframe.
    geometry_XPV = [Point(xy) for xy in zip(data_XPV_meta.Longitude.values.astype('float64'),data_XPV_meta.Latitude.values.astype('float64'))]
    geo_data_XPV = gpd.GeoDataFrame(data_XPV_meta, crs = 'crs', geometry = geometry_DPV)
    return geo_data_XPV
    

-----

In [17]:
#visualización de las plantas

def plants_visualisation(geo_data_XPV, map_precision = 'states', BBox = (-125.00, -66.00, 24.20, 49.50), tech = 'UPV'):
    '''
    plants_visualisation(geo_data_XPV, map_precision = 'states', BBox = (-125.00, -66.00, 24.20, 49.50), tech = 'UPV')
    
    Creates a plot within the bounding box BBox which has coordinates.
    '''
    #Lectura de archivo shape para generación del mapa.
    if map_precision == 'nation':
        usa_map = gpd.read_file('./usa_shapes/usa_shape_nation/cb_2018_us_nation_5m.shp')
    if map_precision == 'states':
        usa_map = gpd.read_file('./usa_shapes/usa_shape_states/cb_2018_us_state_500k.shp')
    if map_precision == 'countys':
        usa_map = gpd.read_file('./usa_shapes/usa_shape_countys/cb_2018_us_county_500k.shp')
        
    if tech == 'UPV':
        colmap = 'YlOrBr'
    elif tech == 'DPV':
        colmap = 'Blues'
    
    fig,ax = plt.subplots(figsize = (40,15))
    usa_map.plot(ax = ax, alpha = 1, color='black')
    ax.set_title('Plant distribution on the USA')
    ax.set_xlim(BBox[0],BBox[1])
    ax.set_ylim(BBox[2],BBox[3])
    power = geo_data_XPV['Power (MW)'].astype('float64')
    geo_data_XPV.plot(ax = ax, markersize = 25, column = power, c = power, cmap= colmap ,marker='o', label = tech, alpha = 0.7 ,legend=True)
    plt.legend()
    plt.show()
    


In [9]:
def coordinate_points(geo_data_XPV):
    '''
    coordinate_points(geo_data_XPV)
    
    Creates 2d array with points coordinates from the given geo_dataframe.
    '''
    geo_points = geo_data_XPV.geometry.values
    points = [ np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geo_points ]
    return points

Segmentation with K means


In [14]:
def geographical_plant_clustering(geo_data_XPV, N_clusters = 50 ):
    '''
    geographical_plant_clustering(geo_data_XPV, N_clusters = 50 )
    
    Clusters from the geographical coordinates of the plants. The funtion assigns a clustering class to each one of 
    the plants on the data frame and creates another data frame with the segmented centroids. In the second data 
    frame, it appends the nearest city with its own coordinates as well.
    '''
    #clustering
    points = coordinate_points(geo_data_XPV)
    kmeans = KMeans(n_clusters = N_clusters, init ='k-means++')
    kmeans.fit(points) # Compute k-means clustering.
    geo_data_XPV['cluster_label_kmeans'] = kmeans.fit_predict(points) # Labels of each point on geo_dataframe
    centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
    centers_df = pd.DataFrame(centers, columns = ['center_long','center_lat'])

    #calling cities database
    us_cities = pd.read_csv('./files/uscities.csv')
    us_cities = us_cities.filter(['city','state_id','lat','lng'], axis=1)
    us_cities = us_cities.rename(columns={"lat": "Latitude", "lng": "Longitude"})
    us_cities.head()

    #appending nearest city from segmentation centers to centers dataframe
    cities = []
    states = []
    lat = []
    lng = []
    for x1,y1 in zip(centers_df.center_long.values, centers_df.center_lat.values):
        distances = []
        for x2,y2 in zip(us_cities.Longitude.values, us_cities.Latitude.values):
            a = np.array([x1,y1])
            b = np.array([x2,y2])
            distances.append(np.linalg.norm(a-b))

        ind = distances.index(min(distances))
        cities.append(us_cities.values[ind][0])
        states.append(us_cities.values[ind][1])
        lat.append(us_cities.values[ind][2])
        lng.append(us_cities.values[ind][3])

    #adds to dataframe data from nearest city to the centroid
    centers_df['Nearest_city'] = cities
    centers_df['State_near_city'] = states
    centers_df['city_long'] = lng
    centers_df['city_lat'] = lat
    
    geo_data_XPV_labeled = geo_data_XPV
    
    return geo_data_XPV_labeled, centers_df

In [1]:
def plant_cluster_plotting(geo_data_XPV, centers_df, map_precision = 'states', BBox = (-125.00, -66.00, 24.20, 49.50), tech = 'UPV', coords = None):
    '''
    plant_cluster_plotting(geo_data_XPV, centers_df, map_precision = 'states', BBox = (-125.00, -66.00, 24.20, 49.50), tech = 'UPV', coords = None)
    
    Plots the different resulting clusters from the segmentation process.
    '''
    
    #Lectura de archivo shape para generación del mapa.
    if map_precision == 'nation':
        usa_map = gpd.read_file('./usa_shapes/usa_shape_nation/cb_2018_us_nation_5m.shp')
    if map_precision == 'states':
        usa_map = gpd.read_file('./usa_shapes/usa_shape_states/cb_2018_us_state_500k.shp')
    if map_precision == 'nation':
        usa_map = gpd.read_file('./usa_shapes/usa_shape_countys/cb_2018_us_county_500k.shp')
        
    if tech == 'UPV':
        colmap = 'YlOrBr'
    elif tech == 'DPV':
        colmap = 'Blues'
        
    fig,ax = plt.subplots(figsize = (40,15))
    usa_map.plot(ax = ax, alpha = 1, color='gray')
    ax.set_title('Plant clusters on USA')
    ax.set_xlim(BBox[0],BBox[1])
    ax.set_ylim(BBox[2],BBox[3])
    power = geo_data_XPV['Power (MW)'].astype('float64')
    cluster = geo_data_XPV['cluster_label_kmeans'].astype('float64')
    geo_data_XPV.plot(ax = ax, markersize = 25, column = cluster, c = cluster, cmap= colmap ,marker='o', label = tech + ' groups', alpha = 0.75 )
    plt.scatter(centers_df.center_long, centers_df.center_lat, c='black', s=200, alpha=0.5)
    if coords is not None:
        coords_df = pd.Dataframe( data = coords, coluns = ['long', 'lat'])
        plt.scatter(coords_df.long, coords_df.lat, c='yellow', s=175, alpha=0.7)
    plt.legend()
    plt.show()
    

In [20]:
def closest_centroid(geo_data_XPV_labeled, centers_df, coords = (-120,25)):
    '''
    closest_centroid(geo_data_XPV_labeled, centers_df, coords = (-120,25))
    
    Calculates the nearest_centroid from a segmentation process given a coordinate position, 
    all of them given in centers_df. Returns the index of the nearest centroid, and its coordinates.
    '''
    long = coords[0]
    latit = coords[1]
    
    distances = []
    for x1,y1 in zip(centers_df.center_long.values, centers_df.center_lat.values):

        a = np.array([coords[0], coords[1]])
        b = np.array([x1,y1])
        distances.append(np.linalg.norm(a-b))
        
    min_ = min(distances)
    max_ = max(distances)

    index_min = distances.index(min_)
    index_max = distances.index(max_)

    center_lng = centers_df['center_long'][index_min]
    center_lat = centers_df['center_lat'][index_min]
    
    lng_max = centers_df['center_long'][index_max]
    lat_max = centers_df['center_lat'][index_max]

    b = np.array([lng_max,lat_max])
    dist = np.linalg.norm(a-b)
    
    if  dist > max_:
        warnings.warn("The selected coordinates are {0} times further than the furthest plant that belongs to the calculated centroid. Climatic variations may differ in a larger propportions to the ideal ones. Take it into account. Proceding...".format(dist/max_)))
    

    return ind, center_lng, center_lat

def cluster_group(index, geo_data_XPV_labeled):
    '''
    cluster_group(index, geo_data_XPV_labeled)
    
    Returns the metadata of only the plants on geo_data_XPV_labeled which are on the cluster group marked with index-
    '''
    
    cluster_group_meta = geo_data_XPV_labeled.loc[geo_data_XPV_labeled['cluster_label_kmeans'] == index] 
    return cluster_group_meta.reset_index(drop=True)


In [ ]:
def read_data(self, geo_data_XPV_labeled = None, path = None, set_points = False):
    '''
    read_data(self, geo_data_XPV_labeled = None, path = None, set_points = False):

    Se leen los metadatos de algún grupo de plantas contenidos en el dataframe o csv indicado en
    geo_data_XPV_labeled o en path según sea el caso. Después se extrae la información de los 
    perfiles de generación, así como de la potencia de las respectivas plantas. 
    Se extraen los valores de generación de la planta al medio día y se devuelve en el array llamado 'points'.
    '''
    if path is not None:
        plants_metadata_df = pd.read_csv(path)
    else:
        plants_metadata_df = geo_data_XPV_labeled
        
    #files = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))] SE LEEN LOS ARCHIVOS EN PATH
    files = geo_data_XPV_labeled['File_name']
    power_plants_MW = geo_data_XPV_labeled['Power (MW)'].astype('float64')
    #for file in files:    SE LEEN LOS NUMEROS QUE TIENEN EL NOMBRE DEL ARCHIVO PARA EXTRAER LA POTENCIA DE LA PLANTA.
     #   power_plants_MW.append(int(''.join(list(filter(str.isdigit, file)))))   
    
    
    self.power_plants_MW = power_plants_MW
    points = []
    
    for file, MW in zip(files, power_plants_MW):
        plant = pd.read_csv('./data/' + file, header=0, index_col=0, parse_dates=True, squeeze=True)
        midday_data = MW*np.ones((364,2))
        for j in range(364):
            i=2*j+1
            midday_data[j][1] = plant[i*int(288/2):i*int(288/2) + 1]
        points.append(midday_data)
    points = np.array(points) 

    if set_points:
        self.points = points

    return points 